# Kaggle: San Francisco Crime Classification

Predict the category of crimes that occurred in the city by the bay

From 1934 to 1963, San Francisco was infamous for housing some of the world's most notorious criminals on the inescapable island of Alcatraz.

Today, the city is known more for its tech scene than its criminal past. But, with rising wealth inequality, housing shortages, and a proliferation of expensive digital toys riding BART to work, there is no scarcity of crime in the city by the bay.

From Sunset to SOMA, and Marina to Excelsior, this competition's dataset provides nearly 12 years of crime reports from across all of San Francisco's neighborhoods. Given time and location, you must predict the category of crime that occurred.

In [1]:
# to add
# probability parameter
# call .decision_function(x) to get probabilities

# normalize your features
# try day of year, day or month, day of week, hour of day...
# polarize features, as done in the M&V 

In [2]:
# %matplotlib qt

In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import cross_validation, preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from os.path import expanduser, normpath
import time
import datetime

In [4]:
# Set paths for data to be imported

home = expanduser('~')
# path = str(home) + '\\Documents\\data-science\\kaggle\\sf-crime\\' # Windows
path = str(home) + '/Documents/Personal/data-science/kaggle/sf-crime/' # Mac
trainfile = 'train.csv'
testfile = 'test.csv'

In [5]:
train_data_raw = pd.read_csv(path+trainfile)
test_data_raw = pd.read_csv(path+testfile)

In [6]:
# train_data.groupby('Category').size().sort_values()

## Features

In [7]:
train_data_raw['Category'].value_counts()

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

In [8]:
train_data = train_data_raw.copy()
test_data = test_data_raw.copy()

In [9]:
# Remove unnecessary features
train_data.drop(['Descript', 
                 'Resolution', 
                 'PdDistrict', 
                 'DayOfWeek', 
                 'Address'], inplace=True, axis=1)

test_data.drop(['PdDistrict', 
                'DayOfWeek', 
                'Address'], inplace=True, axis=1)

In [10]:
train_data['Dates'] = pd.to_datetime(train_data['Dates'])
train_data['year'] = train_data['Dates'].dt.year
train_data['month'] = train_data['Dates'].dt.month 
train_data['day'] = train_data['Dates'].dt.day
train_data['hour'] = train_data['Dates'].dt.hour
train_data['minute'] = train_data['Dates'].dt.minute

train_data['dayofyear'] = train_data['Dates'].dt.dayofyear
train_data['dayofweek'] = train_data['Dates'].dt.dayofweek

In [11]:
test_data['Dates'] = pd.to_datetime(test_data['Dates'])
test_data['year'] = test_data['Dates'].dt.year
test_data['month'] = test_data['Dates'].dt.month 
test_data['day'] = test_data['Dates'].dt.day
test_data['hour'] = test_data['Dates'].dt.hour
test_data['minute'] = test_data['Dates'].dt.minute

test_data['dayofyear'] = test_data['Dates'].dt.dayofyear
test_data['dayofweek'] = test_data['Dates'].dt.dayofweek

In [12]:
# Set up 0 to 1 scaler for preprocessing features in training and test sets
min_max_scaler = preprocessing.MinMaxScaler()

In [13]:
train_data_noscale = pd.DataFrame(train_data[['Category','Dates']])
train_data.drop(['Category', 'Dates'], inplace=True, axis=1)

In [14]:
train_data_minmax = pd.DataFrame(min_max_scaler.fit_transform(train_data),
                                 index = train_data.index, 
                                 columns = train_data.columns) 
train_data_minmax[['Category','Dates']] = train_data_noscale

In [15]:
test_data_noscale= pd.DataFrame(test_data[['Id','Dates']])
test_data.drop(['Id', 'Dates'], inplace=True, axis=1)

In [16]:
test_data_minmax = pd.DataFrame(min_max_scaler.transform(test_data),
                                index = test_data.index, 
                                columns = test_data.columns) 
test_data_minmax[['Id','Dates']] = test_data_noscale

In [17]:
# generate training and cross-validation features
X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(train_data_minmax.ix[:,['dayofyear','dayofweek','hour','X','Y']], 
                                                                 train_data_minmax.ix[:,'Category'], 
                                                                 test_size=0.80, 
                                                                 random_state=1)

In [18]:
X_test = test_data.ix[:,['dayofyear','dayofweek','hour','X','Y']]

In [19]:
# # polarize data
#     if tod:
#         times = index.hour
#         tody = np.cos(2*np.pi*times/24)
#         todx = np.sin(2*np.pi*times/24)     
        
#         X_train[:,2] = tody[shuffling][:n_points]
#         X_train[:,3] = todx[shuffling][:n_points]
        
#         X_test[:,2] = tody[shuffling][n_points:]
#         X_test[:,3] = todx[shuffling][n_points:]

## Classification Model

### Exhaustive Grid Search

In [20]:
# Split again, generate training and cross-validation features for grid search
X_grid_train, X_grid_cv, y_grid_train, y_grid_cv = cross_validation.train_test_split(X_train, 
                                                                 y_train, 
                                                                 test_size=0.50, 
                                                                 random_state=1)

In [ ]:
param_grid = [
    {'estimator__C': [100, 1000, 10000], 'estimator__gamma': [0.01, 0.001, 0.0001], 'estimator__kernel': ['rbf']}
]
scores = ['precision', 'recall']

In [ ]:
%%time
clf = GridSearchCV(OneVsRestClassifier(SVC()), param_grid, error_score=0)
clf.fit(X_grid_train, y_grid_train)

print(clf.best_score_, clf.best_params_)

In [ ]:
# for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     print()

#     clf = GridSearchCV(OneVsRestClassifier(SVC()), param_grid,
#                        scoring='%s_weighted' % score)
#     clf.fit(X_grid_train, y_grid_train)

#     print("Best parameters set found on development set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on development set:")
#     print()
#     for params, mean_score, scores in clf.grid_scores_:
#         print("%0.3f (+/-%0.03f) for %r"
#               % (mean_score, scores.std() * 2, params))
#     print()

#     print("Detailed classification report:")
#     print()
#     print("The model is trained on the full development set.")
#     print("The scores are computed on the full evaluation set.")
#     print()
#     y_true, y_pred = y_grid_cv, clf.predict(X_grid_cv)
#     print(classification_report(y_true, y_pred))
#     print()

In [ ]:
# Run for 5% of sample set, ~1.5 hour run time
# # Tuning hyper-parameters for precision

# {'estimator__C': 1000, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}

# Grid scores on development set:

# 0.140 (+/-0.162) for {'estimator__C': 1, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.093 (+/-0.168) for {'estimator__C': 1, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}
# 0.137 (+/-0.194) for {'estimator__C': 10, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.091 (+/-0.159) for {'estimator__C': 10, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}
# 0.134 (+/-0.116) for {'estimator__C': 100, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.091 (+/-0.160) for {'estimator__C': 100, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}
# 0.154 (+/-0.123) for {'estimator__C': 1000, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.131 (+/-0.186) for {'estimator__C': 1000, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}

# Detailed classification report:

# The model is trained on the full development set.
# The scores are computed on the full evaluation set.

#              precision    recall  f1-score   support

#       ARSON       0.00      0.00      0.00        87
#     ASSAULT       0.00      0.00      0.00      3827
#  BAD CHECKS       0.00      0.00      0.00        19
#     BRIBERY       0.00      0.00      0.00        13
#    BURGLARY       0.00      0.00      0.00      1877
# DISORDERLY CONDUCT       0.00      0.00      0.00       219
# DRIVING UNDER THE INFLUENCE       0.00      0.00      0.00       121
# DRUG/NARCOTIC       0.00      0.00      0.00      2789
# DRUNKENNESS       0.00      0.00      0.00       222
# EMBEZZLEMENT       0.00      0.00      0.00        59
#   EXTORTION       0.00      0.18      0.00        11
# FAMILY OFFENSES       0.00      0.09      0.00        23
# FORGERY/COUNTERFEITING       0.00      0.00      0.00       545
#       FRAUD       0.00      0.00      0.00       846
#    GAMBLING       0.00      0.00      0.00        11
#  KIDNAPPING       0.00      0.00      0.00       118
# LARCENY/THEFT       0.33      0.00      0.00      8772
# LIQUOR LAWS       0.00      0.00      0.00        87
#   LOITERING       0.00      0.00      0.00        62
# MISSING PERSON       0.00      0.00      0.00      1372
# NON-CRIMINAL       0.00      0.00      0.00      4541
# OTHER OFFENSES       0.00      0.00      0.00      6345
# PORNOGRAPHY/OBSCENE MAT       0.00      1.00      0.00         2
# PROSTITUTION       0.00      0.00      0.00       361
# RECOVERED VEHICLE       0.00      0.00      0.00       138
#     ROBBERY       0.00      0.00      0.00      1151
#     RUNAWAY       0.00      0.00      0.00       102
# SECONDARY CODES       0.00      0.00      0.00       485
# SEX OFFENSES FORCIBLE       0.00      0.00      0.00       199
# SEX OFFENSES NON FORCIBLE       0.00      0.00      0.00        10
# STOLEN PROPERTY       0.00      0.00      0.00       225
#     SUICIDE       0.00      0.00      0.00        26
# SUSPICIOUS OCC       0.00      0.00      0.00      1512
#        TREA       0.00      0.00      0.00         0
#    TRESPASS       0.00      0.00      0.00       344
#   VANDALISM       0.00      0.00      0.00      2204
# VEHICLE THEFT       0.00      0.00      0.00      2625
#    WARRANTS       0.00      0.00      0.00      2148
# WEAPON LAWS       0.00      0.00      0.00       404

# avg / total       0.07      0.00      0.00     43902

# # Tuning hyper-parameters for recall

# {'estimator__C': 10, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}

# Grid scores on development set:

# 0.006 (+/-0.005) for {'estimator__C': 1, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.030 (+/-0.036) for {'estimator__C': 1, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}
# 0.002 (+/-0.001) for {'estimator__C': 10, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.033 (+/-0.006) for {'estimator__C': 10, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}
# 0.013 (+/-0.028) for {'estimator__C': 100, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.007 (+/-0.006) for {'estimator__C': 100, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}
# 0.002 (+/-0.005) for {'estimator__C': 1000, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.001}
# 0.000 (+/-0.000) for {'estimator__C': 1000, 'estimator__kernel': 'rbf', 'estimator__gamma': 0.0001}

# Detailed classification report:

# The model is trained on the full development set.
# The scores are computed on the full evaluation set.

#              precision    recall  f1-score   support

#       ARSON       0.00      0.00      0.00        87
#     ASSAULT       0.00      0.00      0.00      3827
#  BAD CHECKS       0.00      0.00      0.00        19
#     BRIBERY       0.00      0.00      0.00        13
#    BURGLARY       0.00      0.00      0.00      1877
# DISORDERLY CONDUCT       0.02      0.02      0.02       219
# DRIVING UNDER THE INFLUENCE       0.00      0.00      0.00       121
# DRUG/NARCOTIC       0.00      0.00      0.00      2789
# DRUNKENNESS       0.00      0.00      0.00       222
# EMBEZZLEMENT       0.00      0.00      0.00        59
#   EXTORTION       0.00      0.00      0.00        11
# FAMILY OFFENSES       0.00      0.00      0.00        23
# FORGERY/COUNTERFEITING       0.00      0.00      0.00       545
#       FRAUD       0.00      0.00      0.00       846
#    GAMBLING       0.00      0.00      0.00        11
#  KIDNAPPING       0.00      0.00      0.00       118
# LARCENY/THEFT       0.33      0.00      0.00      8772
# LIQUOR LAWS       0.00      0.00      0.00        87
#   LOITERING       0.00      0.00      0.00        62
# MISSING PERSON       0.00      0.00      0.00      1372
# NON-CRIMINAL       0.00      0.00      0.00      4541
# OTHER OFFENSES       0.00      0.00      0.00      6345
# PORNOGRAPHY/OBSCENE MAT       0.00      0.00      0.00         2
# PROSTITUTION       0.00      0.00      0.00       361
# RECOVERED VEHICLE       0.00      0.00      0.00       138
#     ROBBERY       0.00      0.00      0.00      1151
#     RUNAWAY       0.00      0.00      0.00       102
# SECONDARY CODES       0.00      0.00      0.00       485
# SEX OFFENSES FORCIBLE       0.00      0.00      0.00       199
# SEX OFFENSES NON FORCIBLE       0.00      0.00      0.00        10
# STOLEN PROPERTY       0.00      0.00      0.00       225
#     SUICIDE       0.00      0.00      0.00        26
# SUSPICIOUS OCC       0.03      0.29      0.06      1512
#    TRESPASS       0.00      0.00      0.00       344
#   VANDALISM       0.05      0.33      0.09      2204
# VEHICLE THEFT       0.00      0.00      0.00      2625
#    WARRANTS       0.00      0.00      0.00      2148
# WEAPON LAWS       0.01      0.39      0.02       404

# avg / total       0.07      0.03      0.01     43902

### SVC Model prediction

In [ ]:
# SUPPORT VECTOR MACHINE
t0 = time.time()

crime_svm = OneVsRestClassifier(SVC(kernel='rbf'))
crime_svm.fit(X_train, y_train)

t1 = time.time()
total = t1-t0
print (total)

In [ ]:
t0 = time.time()

score_train = crime_svm.score(X_train, y_train)

t1 = time.time()
total = t1-t0
print ('Score:', score_train, ', Time:', total) # Score:  0.0880649625019 , Time:  0.27322959899902344

In [ ]:
t0 = time.time()

score_cv = crime_svm.score(X_cv, y_cv)

t1 = time.time()
total = t1-t0
print ('Score:', score_cv, ', Time:', total) # Score:  0.0874252605205 , Time:  1.2154865264892578

In [ ]:
t0 = time.time()

output = crime_svm.predict(X_test)

t1 = time.time()
total = t1-t0
print ('Time:', total) # Time: 0.3079204559326172

In [ ]:
pd.get_dummies(output)

In [ ]:
cat_uniques = train_data_raw.Category.unique()

In [ ]:
headers = 'Id,' + ','.join(sorted(cat_uniques)) + '\n'
f = open('y_test.csv', 'w')
f.write(headers)
for i in xrange(len(y_test)):
    arr = [0] * 39
    arr[int(y_test)]